# 数据与需求
原数据如下，这是一个 Series，从 1 到 24 的等差数列：

In [106]:
import pandas as pd
import numpy as np


In [107]:
ser = pd.Series(range(1, 25))
ser


0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
20    21
21    22
22    23
23    24
dtype: int64

需要转为以下形式。对数列整体等分划为三部分，每个部分有两列，前四个在第一列，后四个在第二列，同时对每份数据指定了同名的索引标签。
# 转为以下数据


In [108]:

"""
      A   B
y
1 A   1   5
  B   2   6
  C   3   7
  D   4   8
2 A   9  13
  B  10  14
  C  11  15
  D  12  16
3 A  17  21
  B  18  22
  C  19  23
  D  20  24
"""


'\n      A   B\ny\n1 A   1   5\n  B   2   6\n  C   3   7\n  D   4   8\n2 A   9  13\n  B  10  14\n  C  11  15\n  D  12  16\n3 A  17  21\n  B  18  22\n  C  19  23\n  D  20  24\n'

# 思路
为了方便后续操作，可以将数据先转为 DataFrame，这样我们就可以新增加辅助列了。由于要对现有数据等份划分，我们可以我是利用 pd.cut() 数据分箱来完成，对它指定箱子数量，可以帮助我们进行平均划分，为了方便识别我们在分箱后对每个箱子的标签进行指定。

接下来，按箱子的标签进行分组，每个组就是要分在一起的数据，我们写一个函数对这些数据进行处理，拼接成想要的数据格式。

处理分组的函数可以这样设计，由于它接收的数据只有一列，我们将它 用 NumPy 的 np.reshape() 进行形状变化，变化后构造为 DataFrame，同时给出列和行的标签值。

In [109]:
ser = ser.to_frame('x')
ser

,x
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [110]:
ser = ser.assign(y=pd.cut(ser.index, bins=3, labels=[*'123']))
ser

,x,y
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,2
9,10,2


In [111]:
ser = ser.groupby(ser["y"])
for k, v in ser:
    vv = np.reshape(v.x.to_numpy(), [4, 2], order="F")
    df = pd.DataFrame(vv, index=[*"ABCD"], columns=[*"AB"])
    print(df)

   A  B
A  1  5
B  2  6
C  3  7
D  4  8
    A   B
A   9  13
B  10  14
C  11  15
D  12  16
    A   B
A  17  21
B  18  22
C  19  23
D  20  24
